In [1]:
import starsim as ss
import pandas as pd
import sciris as sc
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["svg.fonttype"] = "none"
import datetime as dt
import matplotlib as mpl
import seaborn as sns
mpl.rcParams["figure.dpi"] = 300  # Increase resolution inside notebook itself
from matplotlib import colors
import os
import zipfile
import glob
import re
import statsmodels.stats.api as sms
from sklearn.neighbors import KernelDensity
days = lambda x: dt.timedelta(days=x)
%matplotlib inline

Starsim 0.5.1 (2024-05-15) — © 2023-2024 by IDM


In [2]:
import sys
sys.path.remove('c:\\users\\alina.muellenmeister\\documents\\github\\gavi-outbreaks')
sys.path.append(r'c:\\users\\alina.muellenmeister\\documents\\github\\syphilis_analyses')

In [3]:
import stisim as sti
from stisim import calibration

STIsim 0.0.1 (2024-05-15) — © 2024 by IDM


# Data

## Read in calibration data

In [53]:
location = 'zimbabwe'
data_dir = r'C:\Users\alina.muellenmeister\Documents\GitHub\syphilis_analyses\data'
data = pd.read_csv(data_dir + '//' +  f'{location}_calib.csv')
data.index = data["year"]
data

,year,n_alive,hiv.prevalence,hiv.n_infected,hiv.new_infections,hiv.new_deaths,hiv.n_on_art
year,,,,,,,
1990,1990,10432409.00,0.069974,730000,210000,25000.0,0.0
1991,1991,10681008.00,0.085198,910000,220000,33000.0,0.0
1992,1992,10900511.00,0.100913,1100000,220000,43000.0,0.0
1993,1993,11092775.00,0.108179,1200000,210000,53000.0,0.0
1994,1994,11261752.00,0.115435,1300000,200000,63000.0,0.0
1995,1995,11410721.00,0.122692,1400000,180000,74000.0,0.0
1996,1996,11541215.00,0.129969,1500000,160000,84000.0,0.0
1997,1997,11653254.00,0.128719,1500000,150000,94000.0,0.0
1998,1998,11747079.00,0.136204,1600000,140000,100000.0,0.0


# Model Output

In [62]:
zipfiles = os.listdir ("results//Calibration")
calibration_summaries_list = []
timeseries_list = []
for idx, zipfile_name in enumerate(zipfiles): 
    result = ss.Samples("results/Calibration/" + zipfile_name, memory_buffer=True, preload=True)
    calibration_summaries_list.append(result.summary)
    for seed, seed_df in enumerate(result):
        seed_df.insert(0, "scenario", zipfile_name.removesuffix(".zip"))
        seed_df.insert(2, "year", np.floor(seed_df.index))
        timeseries_list.append(seed_df)
    if (idx/len(zipfiles)*100) % 10 == 0:
        print("Done: " + str(round(idx/len(zipfiles), 5)*100) + '%')
print('Done: 100.0%')

calibration_timeseries_df = pd.concat(timeseries_list, axis=0)
calibration_summaries_df = pd.concat(calibration_summaries_list, axis=0)

Done: 0.0%
Done: 10.0%
Done: 20.0%
Done: 30.0%
Done: 40.0%
Done: 50.0%
Done: 60.0%
Done: 70.0%
Done: 80.0%
Done: 90.0%
Done: 100.0%


# Goodness of Fit

In [63]:
# Calculate GOF for a certain parameter
def calc_gof_param(this_scenario_df, param, data_param, groupby_flag=False, as_scalar='sum'):
    if groupby_flag:
        this_scenario_param = this_scenario_df.groupby("year")[param].sum()
    else:    
        this_scenario_param =this_scenario_df[param][::12]
    this_scenario_param.index = np.floor(np.round(this_scenario_param.index, 5))
    data_available_years =data_param.dropna().index
    param_gof = calibration.compute_gof(this_scenario_param.loc[data_available_years].values, data_param.dropna().values, as_scalar=as_scalar)
    return param_gof

In [64]:
gof_estimates_list = []
# Loop over considered scenarios
pd.options.mode.chained_assignment = None 
for idx, scenario in enumerate(np.unique(calibration_timeseries_df['scenario'])):
    this_scenario_df = calibration_timeseries_df[calibration_timeseries_df['scenario'] == scenario]

    this_row = pd.DataFrame()
    
    # Combine seeds to a single metric using mean 
    this_scenario_df_combined = pd.DataFrame()
    this_scenario_df_combined.loc[:, "pop_size_pt"] = this_scenario_df.groupby("t")["n_alive"].mean()
    this_scenario_df_combined.loc[:, "plhiv_pt"] = this_scenario_df.groupby("t")["hiv.n_infected"].mean()
    this_scenario_df_combined.loc[:, "hiv_prev_pt"] = this_scenario_df.groupby("t")["hiv.prevalence"].mean()
    this_scenario_df_combined.loc[:, "new_infections_pt"] = this_scenario_df.groupby("t")["hiv.new_infections"].mean()
    this_scenario_df_combined.loc[:, "new_deaths_pt"] = this_scenario_df.groupby("t")["hiv.new_deaths"].mean()
    this_scenario_df_combined.insert(0, "year", np.floor(this_scenario_df_combined.index).astype(int))
    this_row.loc[0, 'scenario'] = scenario
    # Add values of parameters for this scenario: 
    for param in this_scenario_df.iloc[:, -18:].columns:
        this_scenario_df_combined.loc[:, param] = this_scenario_df[param].iloc[0]
        if not param.__contains__('beta'):
            this_row.loc[0, param] = this_scenario_df[param].iloc[0]
   
    # GOF
    this_row["pop_size_gof"] = calc_gof_param(this_scenario_df_combined, "pop_size_pt", data["n_alive"], groupby_flag=False)
    this_row["hiv_prev_gof"] = calc_gof_param(this_scenario_df_combined, "hiv_prev_pt", data["hiv.prevalence"], groupby_flag=False)
    this_row["plhiv_gof"] =  calc_gof_param(this_scenario_df_combined, "plhiv_pt", data["hiv.n_infected"], groupby_flag=False)
    this_row["new_infections_gof"] = calc_gof_param(this_scenario_df_combined, "new_infections_pt", data["hiv.new_infections"], groupby_flag=True)
    this_row["new_deaths_gof"] = calc_gof_param(this_scenario_df_combined, "new_deaths_pt", data["hiv.new_deaths"], groupby_flag=True)
    this_row['total_sum'] = this_row[["pop_size_gof", "hiv_prev_gof", "plhiv_gof", "new_infections_gof", "new_deaths_gof"]].sum(axis=1)
    
    gof_estimates_list.append(this_row)

gof_estimates = pd.concat(gof_estimates_list, axis=0)
gof_estimates.sort_values(by=['total_sum'])

,scenario,init_prev,duration_on_ART,cd4_start_dist,f0_prob,f1_prob,f2_prob,m0_prob,m1_prob,m2_prob,...,m0_conc,m1_conc,m2_conc,p_pair_form,pop_size_gof,hiv_prev_gof,plhiv_gof,new_infections_gof,new_deaths_gof,total_sum
0,scenario_389,0.07,18.0,800.0,0.70,0.25,0.05,0.78,0.21,0.01,...,0.01,0.4,0.5,0.4,1.363523,6.904642,6.368304,14.484174,11.526028,40.646671
0,scenario_715,0.07,18.0,800.0,0.60,0.35,0.05,0.78,0.21,0.01,...,0.01,0.4,0.5,0.6,1.200811,8.541141,7.765171,15.432987,10.040846,42.980957
0,scenario_57,0.07,18.0,800.0,0.85,0.14,0.01,0.78,0.21,0.01,...,0.01,0.4,0.5,0.4,1.482529,7.524316,6.221688,14.883544,13.812116,43.924194
0,scenario_377,0.07,18.0,800.0,0.70,0.25,0.05,0.78,0.21,0.01,...,0.01,0.4,0.5,0.4,1.482426,7.497703,6.121960,15.619468,13.493511,44.215068
0,scenario_699,0.07,18.0,800.0,0.60,0.35,0.05,0.78,0.21,0.01,...,0.01,0.4,0.5,0.6,1.401889,7.440818,6.737585,14.972891,13.736942,44.290125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,scenario_321,0.07,18.0,800.0,0.70,0.25,0.05,0.78,0.21,0.01,...,0.01,0.4,0.5,0.4,1.748352,38.838924,48.929823,65.123580,19.617725,174.258405
0,scenario_328,0.07,18.0,800.0,0.70,0.25,0.05,0.78,0.21,0.01,...,0.01,0.2,0.5,0.4,1.751949,38.100160,47.850724,65.762215,21.803616,175.268663
0,scenario_325,0.07,18.0,800.0,0.70,0.25,0.05,0.78,0.21,0.01,...,0.01,0.4,0.5,0.4,1.753709,39.187721,49.439791,65.345178,19.731374,175.457773
0,scenario_4,0.07,18.0,800.0,0.85,0.14,0.01,0.78,0.21,0.01,...,0.01,0.2,0.5,0.4,1.757643,38.844851,48.924036,73.295736,21.778426,184.600692


# Plots

In [49]:
def plot(ax, this_scenario_df, param, data, gof, title, groupby_flag=False):  
    if not groupby_flag:
        ax.fill_between(this_scenario_df.index, this_scenario_df[param + '_low'], this_scenario_df[param + '_high'], **fill_args)
        ax.plot(this_scenario_df.index, this_scenario_df[param][:], color="b", alpha=1)
    else:
        ax.fill_between(np.unique(this_scenario_df["year"]), 
                           this_scenario_df.groupby("year")[param + "_low"].sum(), 
                           this_scenario_df.groupby("year")[param + "_high"].sum(), **fill_args)
        ax.plot(np.unique(this_scenario_df["year"]), this_scenario_df.groupby("year")[param].sum()[:], color="b", alpha=1)
    
    ax.scatter(data["year"], data[param], color="tab:red")
    ax.set_title(title + ', GOF ' + str(gof))

In [65]:
low = 0.25
high = 0.75

for idx, scenario in enumerate(np.unique(calibration_timeseries_df['scenario'])):
    # Check if plot already exists
    if not os.path.exists("figures" + '//' + str(scenario) + ".png"):
        this_scenario_df = calibration_timeseries_df[calibration_timeseries_df['scenario'] == scenario]
    
        # Prepare the Data Frame by adding quantiles
        this_scenario_df_combined = pd.DataFrame()
        for param in ["n_alive", "hiv.n_infected", "hiv.prevalence", "hiv.new_infections", "hiv.new_deaths", "hiv.n_on_art"]:
            this_scenario_df_combined.loc[:, param] = this_scenario_df.groupby("t")[param].mean()
            this_scenario_df_combined.loc[:, param + '_low'] = this_scenario_df.groupby("t")[param].quantile(q=low)
            this_scenario_df_combined.loc[:, param + '_high'] = this_scenario_df.groupby("t")[param].quantile(q=high)
        this_scenario_df_combined.insert(0, "year", np.floor(this_scenario_df_combined.index).astype(int))
        
        ################################################################################################################################################################
        # Plots -----------------------------------------------------------------------------------------------------------------------------------------------------------
        ################################################################################################################################################################
        fig, ax = plt.subplots(2, 4)
        fig.set_size_inches(20, 12)
        ax = ax.ravel()
        fill_args = {"alpha": 0.3}
    
        # Population
        pop_gof = round(gof_estimates[gof_estimates['scenario'] == scenario]["pop_size_gof"][0], 2)
        plot(ax[0], this_scenario_df_combined, 'n_alive', data, pop_gof, 'Population', groupby_flag=False)

        # PLHIV
        plhiv_gof = round(gof_estimates[gof_estimates['scenario'] == scenario]["plhiv_gof"][0], 2)
        plot(ax[1], this_scenario_df_combined, 'hiv.n_infected', data, plhiv_gof, 'PLHIV', groupby_flag=False)

        # Prevalence
        hiv_prev_gof = round(gof_estimates[gof_estimates['scenario'] == scenario]["hiv_prev_gof"][0], 2)
        plot(ax[2], this_scenario_df_combined, 'hiv.prevalence', data, hiv_prev_gof, 'HIV Prevalence', groupby_flag=False)
    
        # New Infections
        new_infections_gof = round(gof_estimates[gof_estimates['scenario'] == scenario]["new_infections_gof"][0], 2)
        plot(ax[3], this_scenario_df_combined, 'hiv.new_infections', data, new_infections_gof, 'New Infections', groupby_flag=True)
    
        # New Deaths
        new_deaths_gof = round(gof_estimates[gof_estimates['scenario'] == scenario]["new_deaths_gof"][0], 2)
        plot(ax[4], this_scenario_df_combined, 'hiv.new_deaths', data, new_infections_gof, 'New Deaths', groupby_flag=True)
    
        # Number of people on ART 
        plot(ax[5], this_scenario_df_combined, 'hiv.n_on_art', data, new_infections_gof, 'Number of people on ART', groupby_flag=False)
       
        # Paramater Table 
        ax[6].set_axis_off() 
        ax[7].set_axis_off() 
        variables = ["ss_hiv_beta_m2f", "ss_hiv_beta_fm2", "maternal_hiv_beta",  'f0_prob', 'f1_prob', 'f2_prob', "f1_conc", "m1_conc", "p_pair_form"]
        colors = [["#56b5fd"] if param in variables else ["w"] for param in this_scenario_df.iloc[:, -19:].columns]
        ax[7].table(rowLabels=['ss_hiv_beta_m2f', 'ss_hiv_beta_f2m', 'maternal_hiv_beta'] + this_scenario_df.iloc[:, -16:].columns.tolist(),
                    colLabels=["Value"], 
                    colWidths = [0.4], loc='center',
                    cellColours = colors,
                    cellText=[[round(np.mean(this_scenario_df['ss_hiv_beta_m2f']), 5)], 
                              [round(np.mean(this_scenario_df['ss_hiv_beta_fm2']), 5)], 
                              [round(np.mean(this_scenario_df['maternal_hiv_beta']), 5)]
                             ] + [[value] for value in this_scenario_df.iloc[:, -16:].iloc[0]])
    
        # ----------------------------------------------------------------------------------------------------------------------------------------------------------------
        fig.suptitle('Sum GOF: ' + str(round(gof_estimates[gof_estimates['scenario'] == scenario]["total_sum"][0], 2)))
        plt.savefig("figures" + '//' + str(scenario) + ".png", dpi=100)
        plt.close()